In [ ]:
import pandas as pd
from geopy.geocoders import MapBox
from tqdm.auto import tqdm

tqdm.pandas()

geolocator = MapBox(api_key="")

leases = pd.read_csv("Leases.csv")

full_addresses = (
    leases["address"]
    + ", "
    + leases["city"]
    + ", "
    + leases["state"]
    + " "
    + leases["zip"].apply(lambda x: str(int(x)) if pd.notna(x) else "")
)

address_info = pd.DataFrame.from_dict(
    {"id": leases["building_id"], "address": full_addresses}
).drop_duplicates()

In [16]:
def geocode(address):
  location = geolocator.geocode(address, exactly_one=True)
  if location is None:
    return pd.Series([None, None])

  return pd.Series([location.latitude, location.longitude])

address_info[['latitude', 'longitude']] = address_info['address'].progress_apply(lambda x: geocode(x))

address_info.to_csv('address_info.csv')

100%|██████████| 21077/21077 [45:29<00:00,  7.72it/s] 


In [11]:
address_info

,id,address,latitude,longitude
0,Atlanta_Central Perimeter_Atlanta_10 Glenlake ...,"10 Glenlake Pky NE, Atlanta, GA 30328",33.936269,-84.361239
1,Atlanta_Northwest_Atlanta_100 City View_3330 C...,"3330 Cumberland Blvd, Atlanta, GA 30339",33.873792,-84.458516
2,Atlanta_Northwest_Atlanta_1000 Parkwood_1000 P...,"1000 Parkwood Cir SE, Atlanta, GA 30339",33.901861,-84.459599
3,Atlanta_Northwest_Atlanta_1100 Circle 75_1100 ...,"1100 Circle 75 Pky SE, Atlanta, GA 30339",33.889732,-84.465435
4,Atlanta_Central Perimeter_Atlanta_1200 Ashwood...,"1200 Ashwood Pky, Atlanta, GA 30338",33.933749,-84.340570
